In [1]:
def create_best_lstm_model(best_params):
    model = Sequential()
    n_layers = best_params['lstm_n_layers']
    units = best_params['lstm_units']
    for i in range(n_layers):
        model.add(LSTM(
            units=units,
            return_sequences=(i < n_layers - 1),
            input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]) if i == 0 else None
        ))
        model.add(Dropout(rate=best_params['lstm_dropout']))
    model.add(Dense(1))
    learning_rate = best_params['lstm_learning_rate']
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

def create_lstm_model(trial):
    model = Sequential()
    n_layers = trial.suggest_int('lstm_n_layers', 2, 4)  # Optimize the number of LSTM layers
    units = trial.suggest_categorical('lstm_units', [50, 100, 150, 200])
    for i in range(n_layers):
        model.add(LSTM(
            units=units,
            return_sequences=(i < n_layers - 1),
            input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]) if i == 0 else None
        ))
        dropout_rate = trial.suggest_categorical('lstm_dropout', [0.1, 0.2, 0.3, 0.4, 0.5])
        model.add(Dropout(rate=dropout_rate))  # Use specific Dropout rate
    model.add(Dense(1))
    learning_rate = trial.suggest_categorical('lstm_learning_rate', [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001])  # Optimize learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Define optimization objective function
def objective_LSTM(trial):
    num_samples = X_train_lstm.shape[0]*0.8
    batch_size_candidates = [16, 32, 64, 128, 256, 512,1024,2048]#
    batch_size_candidates = [bs for bs in batch_size_candidates if bs <= num_samples]
    param = {
        'epochs': trial.suggest_categorical('lstm_epochs', [10, 20, 30, 40, 50, 75, 100, 150, 200]),#
        'batch_size': trial.suggest_categorical('lstm_batch_size', batch_size_candidates)
    }
    param['batch_size'] = min(param['batch_size'], num_samples) # Ensure batch_size does not exceed the number of training samples
    model = KerasRegressor(build_fn=lambda: create_lstm_model(trial), epochs=param['epochs'], batch_size=param['batch_size'], verbose=0) #
    
    
    # Use 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = []
    
    for train_index, val_index in kf.split(X_train_lstm):
        X_train, X_val = X_train_lstm[train_index], X_train_lstm[val_index]
        y_train_fold, y_val = y_train[train_index], y_train[val_index]
        num_samples = X_train.shape[0]
        # The 'patience' parameter defines how many epochs the model continues training without improvement before stopping
        # If 'restore_best_weights' is True, the model's weights will be restored to the point with the lowest validation loss
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        model.fit(X_train, y_train_fold, validation_data=(X_val, y_val), callbacks=[early_stopping, KerasPruningCallback(trial, 'val_loss')])
        
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
        mse_scores.append(mse)
    
    return np.mean(mse_scores)




def plt_scatter(data,filename):
    dataframes = data[['predicts','records']]
    dataframes1 = dataframes
    r2 = r2_score(dataframes['records'], dataframes['predicts'])  
    # Calculate NRMSE  
    mse = mean_squared_error(dataframes['records'], dataframes['predicts'])  
    nrmse = np.sqrt(mse) / np.mean(dataframes['records'])  
    rrmse = calculate_rrmse1(dataframes['records'], dataframes['predicts'])
    
    mape = mean_absolute_percentage_error(dataframes['records'], dataframes['predicts'])
    acc = calculate_acc(dataframes['records'], dataframes['predicts'])
    plt.figure(figsize=(10, 6))
    plt.hexbin(dataframes['records'], dataframes['predicts'], gridsize=50, cmap='viridis', mincnt=1)
    cb = plt.colorbar(label='Density')
    fit_params = np.polyfit(dataframes['records'], dataframes['predicts'], 1)
    fit_line = np.polyval(fit_params, dataframes['records'])
    plt.plot(dataframes['records'], fit_line, color='red', label='Fit line')# Add 1:1 line
    max_val = max(max(dataframes['records']), max(dataframes['predicts']))
    plt.plot([0, max_val], [0, max_val], linestyle='--', color='green', label='1:1 line')
    plt.title('')
    plt.xlabel('records')
    plt.ylabel('predicts')
    plt.grid(True)
    plt.text(0.02, 0.95, f'R² = {r2:.2f}', transform=plt.gca().transAxes, fontsize=12, va='top')  
    plt.text(0.02, 0.90, f'ACC = {acc:.2f}', transform=plt.gca().transAxes, fontsize=12, va='top')  
    plt.text(0.02, 0.85, f'RRMSE = {rrmse:.2f}%', transform=plt.gca().transAxes, fontsize=12, va='top')  
    plt.text(0.02, 0.80, f'MAPE = {mape*100:.2f}%', transform=plt.gca().transAxes, fontsize=12, va='top')  
    
    # Display the figure  
    plt.savefig(filename)
    plt.show()


def extract_selected_variables(inputpath_base):
    inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '04_selectFeatures','selectFeatures.txt')
    # Construct file path
    gs_infornamtion = pd.read_csv(inpath_dates, sep='\t', header=None)
    gs_infornamtion.columns = ['slected_dynamic_features', 'slected_static', 'regionID']
    gs_infornamtion['slected_dynamic_features'] = gs_infornamtion['slected_dynamic_features'].apply(ast.literal_eval)
    gs_infornamtion['slected_static'] = gs_infornamtion['slected_static'].apply(ast.literal_eval)
    return gs_infornamtion

def calculate_rrmse1(y_true, y_pred):
    """
    Calculate RRMSE (Relative Root Mean Square Error) using the mean of actual y values as reference
    
    Parameters:
    y_true -- Array or list of true values
    y_pred -- Array or list of predicted values
    
    Returns:
    rrmse -- Relative Root Mean Square Error (in percentage)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
    
    # Calculate mean of true values
    mean_y_true = np.mean(y_true)
    
    # Calculate RRMSE
    rrmse = (rmse / mean_y_true) * 100
    
    return rrmse
    
def calculate_rrmse2(y_true, y_pred):
    """
    Calculate rRMSE (Relative Root Mean Square Error) using each actual y value as reference
    
    Parameters:
    y_true -- Array or list of true values
    y_pred -- Array or list of predicted values
    
    Returns:
    rrmse -- Relative Root Mean Square Error (in percentage)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate rRMSE
    rrmse = np.sqrt(np.mean(((y_true - y_pred) / y_true) ** 2)) * 100
    
    return rrmse
# Define custom nRMSE evaluation function
def calculate_nrmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    nrmse = rmse / (y_true.max() - y_true.min())
    return nrmse* 100

def calculate_acc(y_true, y_pred):
    # Calculate means of observed and predicted values
    mean_true = np.mean(y_true)
    mean_pred = np.mean(y_pred)
    
    # Calculate anomalies
    anomaly_true = y_true - mean_true
    anomaly_pred = y_pred - mean_pred
    
    # Calculate ACC
    numerator = np.sum(anomaly_true * anomaly_pred)
    denominator = np.sqrt(np.sum(anomaly_true**2) * np.sum(anomaly_pred**2))
    
    acc = numerator / denominator
    return acc

In [2]:
import os
import sys
import ast
import pandas as pd
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from optuna_integration.keras import KerasPruningCallback
from optuna.samplers import TPESampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm
import tensorflow.keras.backend as K
import json

In [3]:
root_directory = os.getcwd()[0:3]
'''
# Required packages to install
Pre-installed packages:
os, sys, warnings, optuna 

Packages to install:
matplotlib, sklearn, optuna_integration, tensorflow, scikeras, ast

Installation commands:
conda install optuna matplotlib sklearn optuna_integration tensorflow scikeras ast -c pytorch
conda install matplotlib tensorflow scikeras -c pytorch

'''


# Get current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)
 
# Get the name of the current folder
current_folder_name = os.path.basename(current_directory)[0:8]
print("Current folder name:", current_folder_name)
 
# Get the name of the parent folder
parent_directory = os.path.dirname(current_directory)
parent_folder_name = os.path.basename(parent_directory)
print("Parent folder name:", parent_folder_name)

# Variables that need to be modified
crop = parent_folder_name;countryID =current_folder_name;variable = 'mx2t6';
country = countryID.split('_')[1]

############## Region settings #############################################
inpath_dates_other = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID+'\\'+'01_data'+'\\'+'07_Information'
other_infornamtion = pd.read_csv(os.path.join(inpath_dates_other,'information.txt'), sep=' ', header=None)
startyear,endyear,shp_name = other_infornamtion.iloc[0,0],other_infornamtion.iloc[0,1],other_infornamtion.iloc[0,2]

years = range(startyear,endyear+1)
years_str = [str(year) for year in years]
modelname = 'LSTM'
yield_type = 'actual_yield';
root_directory = os.getcwd()[0:3]
inputpath_base = root_directory + '\\SCI\\SCI9_1\\02_data\\'+crop+'\\'+countryID
SelFeature_infornamtion = extract_selected_variables(inputpath_base)
region='I'
institution = 'ECMWF';country =countryID[3:];yield_type= 'actual_yield';
regions = ['I'];Forecastyear = endyear;# Define the best forecast year for each region
Forecastyears = {
    'I': Forecastyear}

当前工作目录: F:\SCI\SCI9_1\01_code\02_Wheat\06_India-test
当前文件夹名字: 06_India
上一级文件夹名字: 02_Wheat


In [ ]:
########### Hyperparameter Tuning ###########################################

################### Load Selected Feature Variables #########################################
Forecastyear = Forecastyears[region]
data= pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','03_modeldata',region+'_data_ori.csv'))
data = data.drop_duplicates(subset=['year', 'idJoin'], keep='first')
data = data[data['year']!=Forecastyear] # Exclude data from the forecast year

TimeFeatures_sel, Static_sel, regionID = SelFeature_infornamtion[SelFeature_infornamtion['regionID'] == region].iloc[0]
feature_all = TimeFeatures_sel+Static_sel
filtered_columns = [col for col in data.columns if any(feature in col for feature in feature_all)]
filtered_columns = [col for col in filtered_columns if 'year.1' not in col] # Try excluding yield-related features
#### Exclude yield data from previous years #########################
filtered_columns = [col for col in filtered_columns if 'Yield' not in col] # Try excluding yield-related features
Static_sel= [col for col in Static_sel if 'Yield' not in col] # Try excluding yield-related features



MLdata_reduced = data[filtered_columns+[yield_type]]
MLdata_reduced['year'] = data['year']
# Load selected weeks
inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '02_extractdates','gs_three_periods.txt')
gs_infornamtion = pd.read_csv(inpath_dates, delim_whitespace=True, header=None)
gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point', 'VI_select2','regionID']
start_point, peak, harvest_point, VI_select2, region = gs_infornamtion[gs_infornamtion['regionID'] == region].iloc[0]

############################ Data Preparation for Parameter Tuning - Data Standardization ###########################################################
data_all = MLdata_reduced;X_all = data_all.drop([yield_type], axis=1);

y_all = data_all[yield_type];
# Data standardization
scaler_X = StandardScaler().fit(X_all)
X = scaler_X.transform(X_all)
scaler_y = StandardScaler().fit(y_all.values.reshape(-1, 1))
y = scaler_y.transform(y_all.values.reshape(-1, 1)).flatten()

X = pd.DataFrame(data=X,columns=X_all.columns.tolist())
if start_point>harvest_point:
    weeks_select_list= list(range(start_point, 47))+list(range(1, harvest_point+1))
else:
    weeks_select_list= list(range(start_point,harvest_point+1))
    
data_list = []
for i in weeks_select_list:
    data_i = X[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
    data_list.append(data_i.values)
# Stack all arrays in data_list into a 3D array
data_list = np.array(data_list)
X_train_lstm = np.transpose(data_list, (1, 0, 2)) # Reshape array to (samples, time steps, features)
y_train = y

################## Create LSTM Model and Perform Hyperparameter Tuning; Skip if Tuning is Completed (Most Time-Consuming); Resume Tuning from Previous Progress if Paused ####################################
storage_name = f"sqlite:///{country}_{modelname}_region{region}.db" # Tuning records will be saved in the folder where your code is located
study_name =f"{country}_{modelname}_region{region}"

# Perform hyperparameter optimization with Optuna and set early stopping mechanism
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), study_name=study_name, storage=storage_name, load_if_exists=True)

# n_jobs indicates parallel computing
study.optimize(objective_LSTM, n_trials=200, n_jobs=4) 

# Output best hyperparameters
print('Best parameters: ', study.best_params)
print('Best score: ', study.best_value)

# Save best hyperparameters as JSON file
with open(f"{study_name}_best_params.json", "w") as f:
    json.dump(study.best_params, f, indent=4)
print("Best parameters have been saved to JSON file")